In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter
from mol_opt.log_mol_opt import format_data_name, load_data, save_data
from mol_opt.task_metrics import measure_task
from otgnn.utils import save_model, load_model, StatsTracker, log_tensorboard
from molgen.metrics.mol_metrics import MolMetrics
from tensorboardX import SummaryWriter
from mol_opt.log_mol_opt import do_epoch
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES
from pathlib import Path

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis


In [12]:
model_type = "pointwise"
sys.argv = ["", "-cuda", "-model_type", model_type]
args = get_args()
outdir_suffix = "dev2"
args.n_epochs = 20 
args.init_model = "{}-test3".format(model_type)
# args.init_model = "{}-test1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 50 

args.penalty_gumbel = False 

args.scale_lambdas = True
args.connectivity = False 
args.valency = False 
args.euler_characteristic_penalty = False
args.conn_lambda_start = 1
args.conn_lambda_end = 30
args.conn_lambda_epochs_start = 12000
args.conn_lambda_epochs_end = 14000
args.valency_lambda_start = 10
args.valency_lambda_end = 50
args.valency_lambda_epochs_start = 12000
args.valency_lambda_epochs_end = 14000
args.euler_lambda_start = 1
args.euler_lambda_end = 20 
args.euler_lambda_epochs_start = 12000
args.euler_lambda_epochs_end = 14000
args.tau_start = 1
args.tau_end = 0.001
args.tau_epochs_start = 1
args.tau_epochs_end = 16000


args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=50, conn_lambda_end=30, conn_lambda_epochs_end=14000, conn_lambda_epochs_start=12000, conn_lambda_start=1, conn_penalty_function='capped_logdet', connectivity=False, connectivity_hard=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=False, euler_lambda_end=20, euler_lambda_epochs_end=14000, euler_lambda_epochs_start=12000, euler_lambda_start=1, ffn_activation='LeakyReLU', init_decoder_model='pointwise-test3_decode', init_model='pointwise-test3', linear_out=False, max_num_atoms=70, model_type='pointwise', n_epochs=20, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=200, n_labels=1, n_layers=5, one_batch_train=False, ot_solver='emd', output_dir='mol_opt/output_dev2/pointwise-test3', pc_hidden=150, penalty_gumbel=False, pred_hidden=150, scale_lambdas=True, sinkhorn_entropy=0.1, sinkhorn_max_it=1000

In [13]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
train_data_loader = get_loader("molgen/data/chembl50", "raw", args.batch_size, same_number_atoms = False)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

In [14]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

Found previous model mol_opt/output_dev2/pointwise-test3/model_pointwise-test3_9, epoch 9. Overwriting args.
Epoch: 10
FGW torch.Size([29508, 5]) 8.44663882162422e-05
FGW torch.Size([29508, 5]) 8.476530638290569e-05
FGW torch.Size([29508, 5]) 8.318025356857106e-05
FGW torch.Size([29508, 5]) 8.440942474408075e-05
FGW torch.Size([29508, 5]) 8.497113594785333e-05
FGW torch.Size([29508, 5]) 8.4500148659572e-05
FGW torch.Size([29508, 5]) 8.171048102667555e-05
FGW torch.Size([29508, 5]) 8.48886338644661e-05
FGW torch.Size([29508, 5]) 8.171058289008215e-05
FGW torch.Size([29508, 5]) 8.348158007720485e-05
FGW torch.Size([29508, 5]) 8.104809967335314e-05
FGW torch.Size([29508, 5]) 8.42040462885052e-05
FGW torch.Size([29508, 5]) 8.258300658781081e-05
FGW torch.Size([29508, 5]) 8.167204941855744e-05
FGW torch.Size([29508, 5]) 8.674088167026639e-05
FGW torch.Size([29508, 5]) 7.465166709152982e-05
FGW torch.Size([29508, 5]) 8.951355266617611e-05
FGW torch.Size([29508, 5]) 7.280457793967798e-05
FGW 

IsADirectoryError: [Errno 21] Is a directory: 'mol_opt/output_dev2/pointwise-test3/data_pointwise-test3_1'

In [5]:
dir_path = os.path.join(args.output_dir, format_data_name(args.init_model, 1))
data_path = os.path.join(dir_path,  "{}_{}.out".format("train", 0))
print(dir_path)
print(data_path)

mol_opt/output_dev2/pointwise-test3/data_pointwise-test3_1
mol_opt/output_dev2/pointwise-test3/data_pointwise-test3_1/train_0.out


In [6]:
loaded_dict = load_data(data_path)

In [7]:
loaded_dict['losses']

({'fgw': 3.595323181152344,
  'conn_penalty': 0.0,
  'val_penalty': 0.0,
  'euler_penalty': 0.0,
  'total': 3.595323181152344},
 {})

In [8]:
loaded_dict["pred_pack"]

(((tensor([13, 16, 13,  ..., 16, 13, 16]),
   tensor([3, 3, 3,  ..., 3, 3, 3]),
   tensor([4, 3, 3,  ..., 3, 3, 4])),
  (tensor([[ 0.0350, -0.0453,  0.0178,  ...,  0.0346,  0.0418,  0.0557],
           [ 0.0379, -0.0453,  0.0105,  ...,  0.0288,  0.0465,  0.0678],
           [ 0.0359, -0.0464,  0.0111,  ...,  0.0294,  0.0486,  0.0723],
           ...,
           [ 0.0301, -0.0401, -0.0013,  ...,  0.0228,  0.0227,  0.0632],
           [ 0.0202, -0.0440,  0.0079,  ...,  0.0279,  0.0306,  0.0590],
           [ 0.0313, -0.0441,  0.0051,  ...,  0.0284,  0.0359,  0.0638]],
          requires_grad=True),
   tensor([[-0.0535, -0.0756, -0.0574,  0.0841,  0.0702],
           [-0.0626, -0.0684, -0.0577,  0.0855,  0.0658],
           [-0.0646, -0.0687, -0.0570,  0.0848,  0.0647],
           ...,
           [-0.0413, -0.0713, -0.0632,  0.0848,  0.0673],
           [-0.0441, -0.0737, -0.0660,  0.0869,  0.0692],
           [-0.0530, -0.0668, -0.0542,  0.0797,  0.0673]], requires_grad=True),
   tensor(

In [9]:
loaded_dict

{'pred_pack': (((tensor([13, 16, 13,  ..., 16, 13, 16]),
    tensor([3, 3, 3,  ..., 3, 3, 3]),
    tensor([4, 3, 3,  ..., 3, 3, 4])),
   (tensor([[ 0.0350, -0.0453,  0.0178,  ...,  0.0346,  0.0418,  0.0557],
            [ 0.0379, -0.0453,  0.0105,  ...,  0.0288,  0.0465,  0.0678],
            [ 0.0359, -0.0464,  0.0111,  ...,  0.0294,  0.0486,  0.0723],
            ...,
            [ 0.0301, -0.0401, -0.0013,  ...,  0.0228,  0.0227,  0.0632],
            [ 0.0202, -0.0440,  0.0079,  ...,  0.0279,  0.0306,  0.0590],
            [ 0.0313, -0.0441,  0.0051,  ...,  0.0284,  0.0359,  0.0638]],
           requires_grad=True),
    tensor([[-0.0535, -0.0756, -0.0574,  0.0841,  0.0702],
            [-0.0626, -0.0684, -0.0577,  0.0855,  0.0658],
            [-0.0646, -0.0687, -0.0570,  0.0848,  0.0647],
            ...,
            [-0.0413, -0.0713, -0.0632,  0.0848,  0.0673],
            [-0.0441, -0.0737, -0.0660,  0.0869,  0.0692],
            [-0.0530, -0.0668, -0.0542,  0.0797,  0.0673]], 

In [3]:
datapoints = [x for x in os.listdir(args.output_dir) if "data" in x]

In [3]:
tb_writer = SummaryWriter(logdir = args.tb_logs_dir)
metrics = MolMetrics(SYMBOLS, FORMAL_CHARGES, BOND_TYPES, False)

In [86]:
losses_stats_tracker.print_stats()
measure_stats_tracker.print_stats()
metrics_stats_tracker.print_stats()


 fgw:0.0066266
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.0066266

 similarity:0.0122219
 penlog:-10.4965227

 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:97.7294686
 batch_molecular_disconnected_validity:88.8888889
 batch_connected_components:21.1133333
 batch_invalid_valency_nodes:87.1787440
 batch_nodes_0degree:19.8422222
 batch_nodes_7plus_degree:0.0622222
 invalid_euler_toofew:99.1111111
 invalid_euler_toomany:0.0000000
 avg_euler_error:-20.8644444
 batch_node_degree:0.3026087


In [78]:
Path(args.tb_logs_dir).mkdir()

In [6]:
do_epoch(1, args.output_dir, tb_writer, metrics)

Epoch 1
train_0.out
fgw 3.595323181152344
conn_penalty 0.0
val_penalty 0.0
euler_penalty 0.0
total 3.595323181152344
Losses Epoch 1, train
 fgw:3.5953232
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:3.5953232
Measure Epoch 1, train
 similarity:0.0000000
 penlog:-100.0000000
Metrics Epoch 1, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:0.0000000
 batch_molecular_disconnected_validity:0.0000000
 batch_connected_components:1.0000000
 batch_invalid_valency_nodes:99.5652174
 batch_nodes_0degree:0.0000000
 batch_nodes_7plus_degree:22.8600000
 invalid_euler_toofew:0.0000000
 invalid_euler_toomany:100.0000000
 avg_euler_error:260.5800000
 batch_node_degree:24.6591304
train_penalty/learning_rate
train_losses/fgw
train_losses/conn_penalty
train_losses/val_penalty
train_losses/euler_penalty
train_losses/total
train_measure/similarity
train_measure/penlog
train_metrics/batch_molecular_validity
train_metrics/batch_co

In [8]:
Path(args.tb_logs_dir).mkdir(parents=True, exist_ok=True)

In [101]:
os.getcwd()

'/home/octav/gitrepos/tum-thesis'

In [102]:
args.tb_logs_dir

'mol_opt/logs_dev2/pointwise-test3'

In [105]:
log_tensorboard(tb_writer, losses_stats_tracker.get_stats(), data_type + "_losses", epochidx)

In [20]:
losses_stats_tracker.get_stats()

NameError: name 'losses_stats_tracker' is not defined

In [7]:
tb_writer.flush()

In [ ]:
fgw_loss(*pred_pack, tau = 1)